In [ ]:
import sys
sys.path.append('..')
import os
import shutil
import torch

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import numpy
%matplotlib notebook

from data import *
from model import Loop
from utils import generate_merlin_wav

from torch.autograd import Variable
from IPython.display import Audio

def plot(data, labels, dict_file):
    labels_dict = dict_file
    labels_dict = {v: k for k, v in labels_dict.iteritems()}
    labels = [labels_dict[x].decode('latin-1') for x in labels]

    axarr = plt.subplot()
    axarr.imshow(data.T, aspect='auto', origin='lower', interpolation='nearest', cmap=cm.viridis)
    axarr.set_yticks(numpy.arange(0, len(data.T)))
    axarr.set_yticklabels(labels, rotation=90)

In [ ]:
data_path = os.path.abspath('../data/vctk/numpy_features')
norm_info = os.path.abspath('../data/vctk/norm_info/norm.dat')
    
train_dataset = NpzFolder(data_path)
valid_dataset = NpzFolder(data_path + '_valid')

In [ ]:
torch.cuda.set_device(1)

checkpoint = '../models/vctk'
weights = torch.load(checkpoint + '/bestmodel.pth')

args = torch.load(checkpoint + '/args.pth')
opt = args[0]
opt.noise = 0

model = Loop(opt)
model.load_state_dict(weights)
model.cuda();
model.eval();

In [ ]:
ID = 5
txt, feat, _ = valid_dataset[8]

txt = Variable(txt.unsqueeze(1), volatile=True).cuda()
feat = Variable(feat.unsqueeze(1), volatile=True).cuda()
spkr = Variable(torch.LongTensor([ID]), volatile=True).cuda()

out, attn = model([txt, spkr], feat)

generate_merlin_wav(out.data.cpu().numpy(), "/tmp/gen", file_basename='test',
                    norm_info_file=norm_info, do_post_filtering=True)
Audio('/tmp/gen/test.wav')

In [ ]:
plot(attn.squeeze().data.cpu().numpy(), txt[:,0].squeeze().data.tolist(), valid_dataset.dict)

In [ ]:
generate_merlin_wav(feat.data.cpu().numpy(), "/tmp/gen", file_basename='test',
                    norm_info_file=norm_info, do_post_filtering=True)
Audio('/tmp/gen/test.wav')